In [1]:
import pandas as pd
import json
import ast
import re
from datetime import datetime
%load_ext autoreload
%autoreload 2

ETL steam games

In [ ]:
steam_games_ruta = 'datos/output_steam_games.json'

# El json se va a leer linea por linea 
filas_json = []
with open(steam_games_ruta) as f:
    for line in f.readlines():
        data = json.loads(line)
        filas_json.append(data)

# Se almacena en un data frame de pandas
df_games = pd.DataFrame(filas_json)
df_games

In [ ]:
# se cambia el tipo de dato de string a bool
df_games['early_access'] = df_games['early_access'].astype(bool)

In [ ]:
# se cambia el tipo de dato de string a flotante y se asigna cero para los casos especiales
df_games['price'] = df_games['price'].apply(lambda x: round(float(x), 3) if str(x).replace('.', '', 1).isdigit() else 0).astype('float32')

In [ ]:
df_games = df_games.explode('genres')
df_games = df_games.dropna(subset=['genres'])
df_games

In [ ]:
df_games.shape()

In [ ]:
df_games = df_games.drop(['specs', 'url', 'reviews_url','tags'], axis=1)
df_games.head(4)

In [ ]:
#   se observa tipo de dato y valores unicos de price
print(df_games['release_date'].dtype)
print("---"*10)
df_games['release_date'].unique()

In [ ]:
# Asumiendo que la columna 'release_date' es del formato 'yyyy-mm-dd'
df_games['release_year'] = df_games['release_date'].apply(lambda x: x.split('-')[0])

# Ahora, 'release_year' contiene solo el año y se elimina la columna 'release_date'
df_games.drop(columns=['release_date'], inplace=True)

In [ ]:
df_games['release_year'].unique()

In [ ]:
# Define una función lambda para extraer el año si el valor contiene las primeras tres letras de un mes
def extract_year(value):
    try:
        # Si el valor se puede dividir en partes y la última parte es un año, se devuelve
        parts = value.split()
        last_part = parts[-1]
        if last_part.isdigit() and len(last_part) == 4:
            return last_part
    except:
        pass
    return pd.NA

# Reemplaza "SOON" y "SOON™" con "Not-Released-Yet"
df_games['release_year'] = df_games['release_year'].replace(['SOON', 'SOON™'], 'Not-Released-Yet')

# Aplica la función lambda a la columna 'release_year'
df_games['release_year'] = df_games['release_year'].apply(extract_year)

In [ ]:
#lo exporto como csv
ruta_df_clean = 'datos/steam_games_clean.csv'
df_games.to_csv(ruta_df_clean, index=False, encoding='utf-8')

ETL user items

In [ ]:
user_items_ruta = 'datos/australian_users_items.json'

# Lista para almacenar los diccionarios JSON de cada línea
data_list = []

# Abrir el archivo y procesar cada línea
with open(user_items_ruta, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

# Crear un DataFrame a partir de la lista de diccionarios
df_items = pd.DataFrame(data_list)

In [ ]:
df_items.head()

In [ ]:
# se normaliza items
df_items = pd.json_normalize(data_list, record_path=['items'], meta=['steam_id','items_count','user_id', 'user_url'] )
df_items

In [ ]:
# Se proceden a eliminar los duplicados manteniendo el primero
df_items = df_items.drop_duplicates(keep='first')

In [ ]:
#borro la columna playtime_2weeks porque no es necesaria en la api
df_items = df_items.drop('playtime_2weeks', axis=1)

In [ ]:
df_items.head()

In [ ]:
#lo exporto como csv
items_clean_ruta = 'datos/user_items_clean.csv'
df_items.to_csv(items_clean_ruta, index=False, encoding='utf-8')

ETL reviews

In [ ]:
user_reviews_ruta = 'datos/australian_user_reviews.json'

# Lista para almacenar los diccionarios JSON de cada línea
data_list = []

# Abrir el archivo y procesar cada línea
with open(user_reviews_ruta, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

# Crear un DataFrame a partir de la lista de diccionarios
df_reviews = pd.DataFrame(data_list)

In [ ]:
df_reviews

In [ ]:
# Se procede a eliminar duplicados dejando el primer review
df_reviews = df_reviews.drop_duplicates(subset='user_id', keep='first')

In [ ]:
# Se transforma el json a un formato tabular en columnas
df_reviews_norm = pd.json_normalize(df_reviews['reviews'])
df_reviews_norm.head()

In [ ]:
# Se concatenan las columnas que se perdieron
df_reviews_norm = pd.concat([df_reviews[['user_id', 'user_url']], df_reviews_norm], axis=1)
df_reviews_norm.head()